# 4 The data

In [1]:
import os
import numpy as np
import pickle
import random
import re

Read both positive and negative reviews and put it into a large python list.

In [2]:
neg_dir = '../data/neg/'
pos_dir = '../data/pos/'
data = []
step = 0
for filename in os.listdir(neg_dir):
#     step+=1
#     if step>2:
#         break
    path = neg_dir+filename
    with open(path) as f:
        text = f.read().replace('\n','')
        example = {}
        example['label'] = -1
        # Strip out the parse information and the phrase labels---we don't need those here
        text = re.sub(r'\s*(\(\d)|(\))\s*', '', text)
        example['text'] = text[1:]
        data.append(example)
        
for filename in os.listdir(pos_dir):
#     step+=1
#     if step>2:
#         break
    path = pos_dir+filename
    with open(path) as f:
        text = f.read().replace('\n','')
        example = {}
        example['label'] = 1
        # Strip out the parse information and the phrase labels---we don't need those here
        text = re.sub(r'\s*(\(\d)|(\))\s*', '', text)
        example['text'] = text[1:]
        data.append(example)

#Shuffle data
random.seed(1)
random.shuffle(data)

In [3]:
#Training-validation split
training_set = data[:1500]
validation_set = data[1500:]

/newpage
# Sparse Representations

My feature extract function transfer the text into both sparse representation dictionary(named'feature') as described in question and bag-of-words representation(named 'vector') numpy vector(with tons of zeros).

In [4]:
#Write a function to extract bag-of-words features
#The feature_function is modified from Samuel Bowman's Natural Language Processing class in fall 2016.
import collections
import numpy as np

def feature_function(datasets):
    '''Annotates datasets with feature vectors.'''
    
    # Extract vocabulary
    def tokenize(string):
        return string.split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    vocabulary = set([word for word in word_counter])
                                
    feature_names = set()
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['features'] = collections.defaultdict(float)
            
            # Extract features (by name) for one example
            word_counter = collections.Counter(tokenize(example['text']))
            for x in word_counter.items():
                if x[0] in vocabulary:
                    example["features"][x[0]] = x[1]
            
            feature_names.update(example['features'].keys())
                            
    # By now, we know what all the features will be, so we can
    # assign indices to them.
    feature_indices = dict(zip(feature_names, range(len(feature_names))))
    indices_to_features = {v: k for k, v in feature_indices.items()}
    dim = len(feature_indices)
                
    # Now we create actual vectors from those indices.
    for dataset in datasets:
        for example in dataset:
            example['vector'] = np.zeros((dim))
            for feature in example['features']:
                example['vector'][feature_indices[feature]] = example['features'][feature]
    return indices_to_features, feature_indices,dim
    
indices_to_features,feature_indices,dim = feature_function([data])

\newpage
# 6 Support Vector Machine Via Pegasos
## 6.1
Since $\frac{\lambda}{2}\lvert\lvert w\rvert\rvert^2$ is convex and the hinge loss is convex, using the fact provided in the question the subgradient of the objective function is:$
\begin{cases}
\lambda w -y_i x_i & 1-y_i w x_i >0\\
\lambda w & else
\end{cases}
$


Using the stochastic subgradient descent algorithm, the $w$ is updated by:
$
\begin{cases}
w=w-\eta_t(\lambda w -y_i x_i)=(1-\eta_t\lambda)w+\eta_ty_ix_i & 1-y_i w x_i >0\\
w=w-\eta\lambda w= (1-\eta_t\lambda)w& else
\end{cases}
$. This is the same as Pegasos algorithm.

\newpage
## 6.2


In [5]:
def pegasos_vector(data,lambda_reg=0.1,max_epochs=1000):
    epoch=0
    t=0
    w=np.zeros(len(data[0]['vector'])) 
    while t<max_steps:
        epoch+=1
        for example in data:
            t+=1.
            eta=1.0/(lambda_reg*t)
            y_i = example['label']
            x_i = example['vector']
            if y_i*np.dot(w,x_i)<1:
                w=(1-eta*lambda_reg)*w+eta*y_i*x_i
            else:
                w=(1-eta*lambda_reg)*w
    return w

In [6]:
def pegasos_dict(data,lambda_reg=0.1,max_epochs=1000):
    epoch=0
    t=1.
    w = collections.defaultdict(float)
    while epoch<max_epochs:
        print('step',str(t))
        epoch+=1
        for example in data:
            t+=1
            eta=1.0/(lambda_reg*t)
            y_i = example['label']
            x_i = example['features']
            w_xi = 0
            ##get w.x_i
            for feature in x_i:
                w_xi += x_i[feature]*w[feature]
            #multiply every w element with (1-eta*lambda_reg)
            w.update((x, y*(1-eta*lambda_reg)) for x, y in w.items())
            if y_i*w_xi<1:
                for feature in x_i:
                    w[feature] += eta*y_i*x_i[feature]
    return w

\newpage
## 6.3
Notice: With my feature extraction method, the sparse representation vector is stored as numpy array rather than dictionary. Hence my pegasos DOES NOT suffer the slow down as described in the question.


First verify that this is equivalent.
$$
\begin{split}
w_{t+1}&=s_{t+1}W_{t+1}\\
&=(1-\eta_t \lambda)s_tW_t+\eta_t y_i x_i\\
&=(1-\eta_t \lambda)w_t+\eta_t y_i x_i
\end{split}.
$$
The following is the implementation of this algorithm:

In [7]:
def pegasos_dict_accelerated(data,lambda_reg=0.1,max_epochs=1000):
    epoch=1 # Want the first t to start with 2
    W = collections.defaultdict(float)
    s_t = 1.
    t=1.
    while epoch<max_epochs:
        epoch+=1
        for example in data:
            t+=1
            eta=1.0/(lambda_reg*t)
            s_t =(1-eta*lambda_reg)*s_t
            y_i = example['label']
            x_i = example['features']
            w_xi = 0
            ##get w.x_i
            for feature in x_i:
                w_xi += x_i[feature]*W[feature]
            if y_i*w_xi<1:
                for feature in x_i:
                    W[feature] += (1/s_t)*y_i*x_i[feature]
        #multiply s_t and W to get w_t
        W.update((x, y*s_t) for x, y in W.items())
    return W

\newpage
## 6.4


In [8]:
# First Verify that 2 method yields the same result
test_w = pegasos_dict(training_set,max_epochs=2)

step 1.0
step 1501.0


In [9]:
test_w_acc = pegasos_dict_accelerated(training_set,max_epochs=2)

In [10]:
test_w_acc['fame'],test_w['fame']

(2.31645569620253, 0.019993335554815136)

/newpage
## 6.5

In [11]:
def evaluate(w,data):
    correct = 0
    for example in data:
        label = example['label']
        w_xi=0
        x_i = example['features']
        w_xi = 0
        ##get w.x_i
        for feature in x_i:
            w_xi += x_i[feature]*W[feature]
        if w_xi<0:
            prediction = -1
        else:
            prediction = 1
        if label ==prediction:
            correct+=1
    return correct/len(data)